# Prediction des resultats des match avec Arbre de Decision

In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import graphviz
# Permet d'afficher les figures directement dans le notebook:
%matplotlib inline
%run ./classes_utiles_learning.ipynb

In [3]:
train_data = np.load("./train_data.npy")
train_labels = np.load("./train_labels.npy")
test_data = np.load("./test_data.npy")
test_labels = np.load("./test_labels.npy")

In [4]:
train_set = LabeledSet(len(train_data[0]))
for i in range(len(train_data)):
    train_set.addExample(train_data[i], [train_labels[i]])

In [5]:
test_set = LabeledSet(len(test_data[0]))
for i in range(len(test_data)):
    test_set.addExample(test_data[i], [test_labels[i]])

In [6]:
def classe_majoritaire(Lset):
    
    classes = [-1, +1]
    frequences = [0, 0]
    
    for i in range(Lset.size()):
        frequences[classes.index(Lset.getY(i))] += 1
    
    if(frequences[0] == frequences[1]):
        return +1
    elif(frequences[0] > frequences[1]):
        return classes[0]
    else:
        return classes[1]

In [7]:
def shannon(probas):
    k = len(probas)
    total = 0.0
    for i in probas:
        if(i == 0):
            continue
            
        total = total + (i * (np.log(i) / np.log(k)))
        
    return np.abs(total)

In [8]:
def entropie(Lset):
    n = Lset.size()
    proba1 = 0
    proba_1 = 0
    
    for i in range(n):
        if(Lset.getY(i) == 1):
            proba1 = proba1 + 1
        else:
            proba_1 = proba_1 + 1
            
    probas = [proba1*1.0/n, proba_1*1.0/n]
    return shannon(probas)

In [9]:
def discretise(LSet, col):
    """ LabelledSet * int -> tuple[float, float]
        col est le numéro de colonne sur X à discrétiser
        rend la valeur de coupure qui minimise l'entropie ainsi que son entropie.
    """
    # initialisation:
    min_entropie = 1.1  # on met à une valeur max car on veut minimiser
    min_seuil = 0.0     
    # trie des valeurs:
    ind= np.argsort(LSet.x,axis=0)
    
    # calcul des distributions des classes pour E1 et E2:
    inf_plus  = 0               # nombre de +1 dans E1
    inf_moins = 0               # nombre de -1 dans E1
    sup_plus  = 0               # nombre de +1 dans E2
    sup_moins = 0               # nombre de -1 dans E2       
    # remarque: au départ on considère que E1 est vide et donc E2 correspond à E. 
    # Ainsi inf_plus et inf_moins valent 0. Il reste à calculer sup_plus et sup_moins 
    # dans E.
    for j in range(0,LSet.size()):
        if (LSet.getY(j)[0] == -1):
            sup_moins += 1
        else:
            sup_plus += 1
    nb_total = (sup_plus + sup_moins) # nombre d'exemples total dans E
    
    # parcours pour trouver le meilleur seuil:
    for i in range(len(LSet.x)-1):
        v_ind_i = ind[i]   # vecteur d'indices
        courant = LSet.getX(v_ind_i[col])[col]
        lookahead = LSet.getX(ind[i+1][col])[col]
        val_seuil = (courant + lookahead) / 2.0;
        # M-A-J de la distrib. des classes:
        # pour réduire les traitements: on retire un exemple de E2 et on le place
        # dans E1, c'est ainsi que l'on déplace donc le seuil de coupure.
        if LSet.getY(ind[i][col])[0] == -1:
            inf_moins += 1
            sup_moins -= 1
        else:
            inf_plus += 1
            sup_plus -= 1
        # calcul de la distribution des classes de chaque côté du seuil:
        nb_inf = (inf_moins + inf_plus)*1.0     # rem: on en fait un float pour éviter
        nb_sup = (sup_moins + sup_plus)*1.0     # que ce soit une division entière.
        # calcul de l'entropie de la coupure
        val_entropie_inf = shannon([inf_moins / nb_inf, inf_plus  / nb_inf])
        val_entropie_sup = shannon([sup_moins / nb_sup, sup_plus  / nb_sup])
        val_entropie = (nb_inf / nb_total) * val_entropie_inf + (nb_sup / nb_total) * val_entropie_sup
        # si cette coupure minimise l'entropie, on mémorise ce seuil et son entropie:
        if (min_entropie > val_entropie):
            min_entropie = val_entropie
            min_seuil = val_seuil
    return (min_seuil, min_entropie)

In [10]:
def divise(Lset, att, seuil):
    the_set1 = LabeledSet(Lset.input_dimension)
    the_set2 = LabeledSet(Lset.input_dimension)
    
    for i in range(Lset.size()):
        if(Lset.getX(i)[att] <= seuil):
            the_set1.addExample(Lset.getX(i), Lset.getY(i))
        else:
            the_set2.addExample(Lset.getX(i), Lset.getY(i))
            
    return the_set1,the_set2

In [11]:
import graphviz as gv

class ArbreBinaire:
    def __init__(self):
        self.attribut = None   # numéro de l'attribut
        self.seuil = None
        self.inferieur = None # ArbreBinaire Gauche (valeurs <= au seuil)
        self.superieur = None # ArbreBinaire Gauche (valeurs > au seuil)
        self.classe = None # Classe si c'est une feuille: -1 ou +1
        
    def est_feuille(self):
        """ rend True si l'arbre est une feuille """
        return self.seuil == None
    
    def ajoute_fils(self,ABinf,ABsup,att,seuil):
        """ ABinf, ABsup: 2 arbres binaires
            att: numéro d'attribut
            seuil: valeur de seuil
        """
        self.attribut = att
        self.seuil = seuil
        self.inferieur = ABinf
        self.superieur = ABsup
    
    def ajoute_feuille(self,classe):
        """ classe: -1 ou + 1
        """
        self.classe = classe
        
    def classifie(self,exemple):
        """ exemple : numpy.array
            rend la classe de l'exemple: +1 ou -1
        """
        if self.est_feuille():
            return self.classe
        if exemple[self.attribut] <= self.seuil:
            return self.inferieur.classifie(exemple)
        return self.superieur.classifie(exemple)
    
    def to_graph(self, g, prefixe='A'):
        """ construit une représentation de l'arbre pour pouvoir
            l'afficher
        """
        if self.est_feuille():
            g.node(prefixe,str(self.classe),shape='box')
        else:
            g.node(prefixe, str(self.attribut))
            self.inferieur.to_graph(g,prefixe+"g")
            self.superieur.to_graph(g,prefixe+"d")
            g.edge(prefixe,prefixe+"g", '<='+ str(self.seuil))
            g.edge(prefixe,prefixe+"d", '>'+ str(self.seuil))
        
        return g

In [12]:
def construit_AD(Lset, epsilon):
    entr = entropie(Lset)
    if(entr <= epsilon):
        c_maj = classe_majoritaire(Lset)
        abr = ArbreBinaire()
        abr.ajoute_feuille(c_maj)
        return abr
    else:
        l_seuil = []
        l_entropie = []
        #On calcule l'entropie pour chaque attribut
        for i in range(Lset.input_dimension):
            seuil,entr = discretise(Lset, i)
            l_seuil.append(seuil)
            l_entropie.append(entr)
        
        #on prend l'attribut qui minimise cette entropie
        ind_min = np.argmin(l_entropie)
        seuil = l_seuil[ind_min]
        
        set_gauche,set_droite = divise(Lset, ind_min, seuil)
        
        abr_gauche = construit_AD(set_gauche, epsilon)
        abr_droite = construit_AD(set_droite, epsilon)
        
        abr = ArbreBinaire()
        abr.ajoute_fils(abr_gauche, abr_droite, ind_min, seuil)
        
        return abr

In [13]:
class ArbreDecision(Classifier):
    # Constructeur
    def __init__(self,epsilon):
        # valeur seuil d'entropie pour arrêter la construction
        self.epsilon= epsilon
        self.racine = None
    
    # Permet de calculer la prediction sur x => renvoie un score
    def predict(self,x):
        # classification de l'exemple x avec l'arbre de décision
        # on rend 0 (classe -1) ou 1 (classe 1)
        classe = self.racine.classifie(x)
        if (classe == 1):
            return(1)
        else:
            return(-1)
    
    # Permet d'entrainer le modele sur un ensemble de données
    def train(self,set):
        # construction de l'arbre de décision 
        self.set=set
        self.racine = construit_AD(set,self.epsilon)

    # Permet d'afficher l'arbre
    def plot(self):
        gtree = gv.Digraph(format='png')
        return self.racine.to_graph(gtree)

## Ma fonction d'arbre de décision fonctionne pas avec un epsilon petit, cela est du au fait que la pile des appels récursifs est limité et qu'on arrive à un moment avec beaucoup d'appels dans la pile ce qui bloque le programme. Pour cela, je vais tester avec le code de l'arbre décision déjà fourni dans la librairie standard sklearn

In [14]:
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf = clf.fit(train_data, train_labels)

In [28]:
total_bon = 0
arr = clf.predict(test_data)
for i in range(len(test_data)):
    if(arr[i] == test_labels[i]):
        total_bon = total_bon + 1
print("accuracy = " + str((total_bon*1.0 / l)*100))

accuracy = 62.26231106777181
